Programming assignment 1

You should submit your python code as a solution via vocareum platform, more detailed instructions will come later. Use the PuLP library https://pypi.org/project/PuLP/ to solve the following problems. Documentation to PuLP can be found here: https://coin-or.github.io/pulp/main/index.html

A few hints

To surpress the messages from the solver, you can use GLPK(msg = 0) (if you use GLPK) or PULP_CBC_CMD(msg=False) (if you use CBC) or a similar option for other solvers as a solver parameter of solve() method. You can use function value() to evaluate any expression involving the variables in the optimal solution. If you need many variables, it might be good to introduce them using an array of their names using dicts() method and use the returned dictionary to access them.

Part 1 (10 points)

Solve the following program:

       min 122x + 143y
subject to:
         x ≥ -10
         y ≤  10
  3x +  2y ≤  10
 12x + 14y ≥ -12.5
  2x +  3y ≥   3
  5x -  6y ≥-100
Required output example:

Optimal solution: x = 0.1 y = -2.3
Objective value: 100.1214
Tight constraints:
1
2
4
So, your program should find the optimal solution, determine its objective value, and identify the tight constraints.
Part 2 (10 points)

Find an optimal mixed strategy of the following game: Both players choose independently a single integer from 1 to 6. Then, the numbers are compared:

If they are equal, there is a draw
If they differ by 1, the player who played the smaller number gets 2EUR from the other player
If they differ by ≥2, the player who played the larger number gets 1EUR from the other player
Note that the game is symmetric and the same strategy is optimal for both players.
Required output example:

x1: 0.2 x2: 0.1 x3: 0.2 x4: 0.1 x5: 0.2 x6: 0.2
Part 3 (10 points)

On some imaginary island, there are 69 companies and there are bilateral contracts between them. The monarch of the island would like to inspect validity of each of these contracts during a single large event. The monarch requires two representatives to represent each contract relationship (they can be both from the same party of the contract or each from a different one). This is of course satisfied by each company sending a single representative, which would require involvement of 69 representatives in total. However, the companies want to find a solution which minimizes the total number of representatives who need to attend the event.

Input file hw1-03.txt contains information about the contracts. Each line corresponds to a single contract and contains identifiers (1-69) of both involved parties separated by a space.

Example output:

representatives from company 1: 1.0
representatives from company 2: 2.0
representatives from company 3: 1.0
...
representatives from company 69: 1.0
Total number of representatives involved: 58
Important: It is not possible to send an arbitrary fraction of a representative. However, it is enough to solve the LP relaxation since it already gives an integral solution. Reasons for this will be explained later in the class.

In [ ]:
import pulp
from pulp import *
import numpy as np

Problem 1 

In [ ]:
#Exercise 1
x = LpVariable("x", -10, None)
y = LpVariable("y", None, 10)
problem = LpProblem("problem", LpMinimize)

problem += 122*x + 143*y

problem += 12 * x + 14 *y >= -12.5 
problem += 3*x + 2*y <= 10
problem += 2*x + 3*y >= 3
problem += 5*x - 6*y >= -100

sol = problem.solve(PULP_CBC_CMD(msg = False))
    
constraints = problem.constraints
tightconstraints = []
for name in constraints.keys(): 
    if constraints.get(name).value()==0: 
        tightconstraints.append(name)


print(f'Optimal solutions: x = {value(x)}  y = {value(y)}')
print(f'Objective value: {problem.objective.value()}')
print('Tight constraints:')
for i in range(len(tightconstraints)): 
    print(tightconstraints[i])


    For problem 2 I propose two alternative solutions

In [ ]:
#Exercise 2
x0 = LpVariable("x0")
x1 = LpVariable("x1",0, None)
x2 = LpVariable("x2",0, None)
x3 = LpVariable("x3",0, None)
x4 = LpVariable("x4",0, None)
x5 = LpVariable("x5",0, None)
x6 = LpVariable("x6",0, None)


prob = LpProblem("Game", LpMaximize)

prob += x0
prob += -2*x2 +x3 +x4 +x5 +x6 -x0 >= 0
prob += 2*x1 -2*x3 + x4 + x5 + x6 -x0 >= 0
prob += -x1 + 2*x2 -2*x4 + x5 + x6 -x0 >= 0
prob += -x1 -x2 +2*x3 -2*x5 + x6 -x0 >= 0
prob += -x1 - x2 - x3 +2*x4 -2*x6 -x0 >= 0
prob += -x1 -x2 -x3 -x4 + 2*x5 -x0 >= 0
prob += x1 + x2+ x3 + x4+ x5 + x6 == 1


solution = prob.solve(PULP_CBC_CMD(msg = 0))
print(f'x1: {value(x1)} x2:{value(x2)} x3:{value(x3)} x4:{value(x4)} x5:{value(x5)} x6:{value(x6)}, x0: {value(x0)}')


#Another method

M = np.zeros((6,6,))
for i in range(6): 
    for j in range(6): 
        if j-i == 1: 
            M[i][j] = 2
        if j-i >= 2:
            M[i][j] = -1
        if j-i == -1: 
            M[i][j] = -2
        if j-i <= -2: 
            M[i][j] = 1
        

x_var = LpVariable.dicts("x", [str(i) for i in range(1,7)], lowBound=0)
x_0 = LpVariable("x_0", None)

LP_game = LpProblem("Game", LpMaximize)
LP_game += x_0
for j in range(6): 
        LP_game += lpSum(M[i][j]*x_var[str(i+1)] for i in range(6)) - x_0 >=0
LP_game += lpSum(x_var[str(i)] for i in range(1,7)) == 1

gamesolution = LP_game.solve(PULP_CBC_CMD(msg = 0))

for v in LP_game.variables():
    print(f' {v.name}: {v.varValue}')


Problem 3

In [ ]:
#Exercise3
f = open("bilateralcontracts.txt", 'r')
contracts_name = f.read().splitlines()
firstcol = []
secondcol = []
for element in contracts_name: 
    firstcol.append(element.split(' ')[0])
    secondcol.append(element.split(' ')[1])

contracts_var = np.hstack((np.array([[firstcol[i]] for i in range(len(firstcol))]),np.array([[secondcol[i]] for i in range(len(secondcol)) ])))

companies_name= list(str(i+100) for i in range(1,70))
companies_var = LpVariable.dicts("cacca",list(str(i+100) for i in range(1,70)), lowBound= 0)
print(companies_var)

representatives = LpProblem("Lazy Island", LpMinimize)
representatives += lpSum(companies_var[name] for name in companies_name)
for i in range(len(secondcol)):   
    representatives += lpSum(companies_var[str(contracts_var[i][j])] for j in range(2)) >=2

monarchdecision =  representatives.solve(PULP_CBC_CMD(msg = 0))
#for v in representatives.variables():
    #print(f'representatives from {v.name}: {v.varValue}')
print(f'Total number of representatives involved: {representatives.objective.value()}')

f.close()